## Imports

Importing libraries need here.


In [1]:
# !pip install pandas==1.3.3
# !pip install requests==2.26.0
# !pip install bs4==4.10.0
# !pip install html5lib==1.1

In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
import html5lib
import requests
import pandas as pd
import json
import glob

## Extract Data Using Web Scraping


The wikipedia webpage https://en.wikipedia.org/wiki/List_of_largest_banks provides information about largest banks in the world by various parameters. Scrape the data from the table 'By market capitalization' and store it in a JSON file.


### Webpage Contents

Gather the contents of the webpage in text format using the `requests` library and assign it to the variable <code>html_data</code>


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_largest_banks"

html_data = requests.get(url).text

In [4]:
html_data[123:156]

'der-disabled vector-feature-stick'

### Scraping the Data

Using the contents and `beautiful soup` load the data from the `By market capitalization` table into a `pandas` dataframe. The dataframe should have the bank `Name` and `Market Cap (US$ Billion)` as column names.  Display the first five rows using head. 


In [5]:
soup = BeautifulSoup(html_data, 'html.parser')

Load the data from the `By market capitalization` table into a pandas dataframe. The dataframe should have the bank `Name` and `Market Cap (US$ Billion)` as column names. Using the empty dataframe `data` and the given loop extract the necessary data from each row and append it to the empty dataframe.


In [6]:
data = pd.DataFrame(columns=["Name", "Market Cap (US$ Billion)"])

columns = []
for row in soup.find_all('tbody')[0].find_all('tr'):
    col = row.find_all('td')
    if len(col)!=0:
        bank_name = col[1].text.strip()
        market_cap = col[2].text.strip()

        #finally we append the data of each row to the table
        data = data.append({"Name":bank_name,"Market Cap (US$ Billion)":market_cap}, ignore_index=True)

Display the first five rows using the `head` function.


In [7]:
data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,491.76
1,Bank of America,266.45
2,Industrial and Commercial Bank of China,219.45
3,Wells Fargo,178.74
4,Agricultural Bank of China,175.69



### Loading the Data

Load the `pandas` dataframe created above into a JSON named `bank_market_cap.json` using the `to_json()` function.


In [8]:
data.to_json('bank_market_cap.json')

### Call the API

 <b> Question 1</b> Using the `requests` library call the endpoint given above and save the text, remember the first few characters of the output: 


## Extract Data Using an API


### Call the API

Using the `requests` library call the endpoint given above and save the text, remember the first few characters of the output: 


In [9]:
url = "http://api.exchangeratesapi.io/v1/latest?base=EUR&access_key=73641918aa03ff1031855237c7de9fd5"
data = json.loads(requests.get(url).text)
rates = data.get('rates', {})

### Save as DataFrame

Using the data gathered turn it into a `pandas` dataframe. The dataframe should have the Currency as the index and `Rate` as their columns. Make sure to drop unnecessary columns.


In [10]:
df = pd.DataFrame.from_dict(rates, orient = 'index', columns = ['Rate'])

### Load the Data

Using the dataframe save it as a CSV names `exchange_rates_1.csv`.


In [11]:
df.to_csv("exchange_rates.csv")

In [12]:
df

,Rate
AED,3.977945
AFN,78.517390
ALL,103.699595
AMD,437.769249
ANG,1.952844
...,...
YER,271.134875
ZAR,20.476154
ZMK,9748.520172
ZMW,29.065640


## Extract


### JSON Extract Function

This function will extract JSON files.


In [13]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [14]:
columns=['Name','Market Cap (US$ Billion)']

In [15]:
def extract():    
    #create empty dataframe
    extracted_data = pd.DataFrame(columns = columns)
    
    #extract json file
    extracted_data = extracted_data.append(extract_from_json('bank_market_cap.json'), ignore_index = True)
    display(extracted_data)
    return extracted_data

In [16]:
extracted_data = pd.DataFrame(columns = columns)
    
#extract json file
extracted_data = extracted_data.append(extract_from_json('bank_market_cap.json'), ignore_index = True)
display(extracted_data)

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,491.76
1,Bank of America,266.45
2,Industrial and Commercial Bank of China,219.45
3,Wells Fargo,178.74
4,Agricultural Bank of China,175.69
5,HDFC Bank,169.84
6,HSBC Holdings PLC,156.13
7,Morgan Stanley,153.05
8,China Construction Bank,151.97
9,Bank of China,150.39


Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [17]:
df = pd.read_csv('exchange_rates.csv')
df = df.set_index('Unnamed: 0')
df.index.name = ''


exchange_rate = df.loc['GBP'][0]
exchange_rate

0.853088

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [18]:
def transform(data, exchange_rate):
    
    
    column_name = 'Market Cap (US$ Billion)'
    if column_name not in data.columns:
        column_name = column_name.strip()
    if column_name in data.columns:
        data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)'] * exchange_rate, 3)
        data = data.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'})
    transformed_data = data
    return transformed_data

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [19]:
def load(transform_data, target_file):
    transform_data.to_csv(target_file)

## Logging Function


Write the basic logging function <code>log</code> to log data:


In [20]:
def log(message):
    timestamp_format = '%Y-%h-%D-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open('log_file.txt', 'a') as f:
        f.write( timestamp + " .    " + message + '\n')
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [21]:
log("ETL Job Started")
log("Extract phase Started")

### Extract


Use the function <code>extract</code>, and print the first 5 rows.


In [22]:
extract()

# Print the rows here
print(extracted_data.head())

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,491.76
1,Bank of America,266.45
2,Industrial and Commercial Bank of China,219.45
3,Wells Fargo,178.74
4,Agricultural Bank of China,175.69
5,HDFC Bank,169.84
6,HSBC Holdings PLC,156.13
7,Morgan Stanley,153.05
8,China Construction Bank,151.97
9,Bank of China,150.39


                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                    491.76
1                          Bank of America                    266.45
2  Industrial and Commercial Bank of China                    219.45
3                              Wells Fargo                    178.74
4               Agricultural Bank of China                    175.69


Log the data as <code>"Extract phase Ended"</code>


In [23]:
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [24]:
log("Transform phase Started")

Use the function <code>transform</code> and print the first 5 rows of the output.


In [25]:
transformed_data = transform(data = extracted_data, exchange_rate = exchange_rate)
# Print the first 5 rows here
transformed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,419.515
1,Bank of America,227.305
2,Industrial and Commercial Bank of China,187.210
3,Wells Fargo,152.481
4,Agricultural Bank of China,149.879


Log data <code>"Transform phase Ended"</code>


In [26]:
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


### Load


Log the following `"Load phase Started"`.


In [27]:
log("Load phase Started")

target_file = 'bank_market_cap_gbp.csv'

Call the load function


In [28]:
load(transformed_data, target_file)

Log the following `"Load phase Ended"`.


In [29]:
log("Load phase Ended")